# NLP case study


The following table summarizes the datasets used throughout this notebook.

| dataset ID | dataset name| is_dialectical | is_MSA (Modern Standard Arabic) | is_balanced | num_of_tweets | num_of_pos_tweets | num_of_neg_tweets |
|-- | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | [arabic-sentiment-twitter-corpus](https://www.kaggle.com/mksaad/arabic-sentiment-twitter-corpus) | Yes | No/minority | Yes | 58,751 | 29,849 | 28,902  
| 2 |[SS2030](https://www.kaggle.com/snalyami3/arabic-sentiment-analysis-dataset-ss2030-dataset ) | Yes - Saudi dialect only | No/Minority | Yes | 4,252 | 2,436 | 1,816 
| 3 |[100k Arabic Reviews](https://www.kaggle.com/abedkhooli/arabic-100k-reviews ) | No/Minority | Yes | Yes | 66,666 | 33,333 | 33,333
| 4 | [ArSAS](https://homepages.inf.ed.ac.uk/wmagdy/resources.htm) | Yes - mixed dialects| No/Minority | Yes | 11,784 | 4,400 | 7,384

*(For a more detailed analysis of the datasets see [this](https://www.kaggle.com/yasmeenhany/dataset-analysis) companion notebook. )*



**[Importing the necessary modules](https://)**

In [1]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import io
import emoji
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

%matplotlib inline

### Importing the dataset


In [3]:
pd.set_option('display.max_colwidth', 280)
pd.set_option('mode.chained_assignment', None)
train_neg = pd.read_csv("twitter/train_Arabic_tweets_negative_20190413.tsv", sep="\t", header=None,  quoting=csv.QUOTE_NONE)
train_pos = pd.read_csv("twitter/train_Arabic_tweets_positive_20190413.tsv", 
                        sep="\t", header=None,  quoting=csv.QUOTE_NONE)
train_neg.rename(columns={0:'label', 1:'tweet'}, inplace=True)
train_pos.rename(columns={0:'label', 1:'tweet'}, inplace=True)
train_neg['label'] = 0
train_pos['label'] = 1
twitter_train_df = pd.concat([train_neg, train_pos], axis=0).reset_index(drop=True)

### Visualizing the first 10 rows of the training dataset: 

In [4]:
twitter_train_df.head(10)

,label,tweet
0,0,اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم بالزايد 😭
1,0,توقعت اذا جات داريا بشوفهم كاملين بس لي للحين احس فيه احد ناقصهم 💔 #Avlu
2,0,#الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال والاهلي تحت التاق 👇 #تحدي_اسرع_روقان وادخل في سحب قيمة ايفون X على…
3,0,نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين على بكتيريا 🦠 فتنفجر 💥 و تموت . الأخيرة يبدو انها بكتيريا مقاومة فأخذ…
4,0,الدودو جايه تكمل علي 💔
5,0,أحببته حتى أقنعنى إن ما فات من العمر كان إنتظار له 🙈
6,0,يبدو ان دجلة اعتادت على التهام اجساد ابنائها من سبايكر للعبارة ..👆👆💔 ما سمعته .. ان البشر يعطش فيشرب الماء .. لم…
7,0,جالس أذاكر 😣
8,0,يكفي استخفاف بالعقل العربي هل حقا الأمن والمخابرات في ألمانيا غير قادرة على وقف اي عصابة أو الجريمة 🤔 وا…
9,0,ياليلل ترا اكلج كثير مشي 😭


In [11]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

### Preprocessing the dataset

In [5]:
import string
punctuations_list = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation
arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    text = text.replace(punctuations_list,' ')
    return text


def remove_emojis(text):
  emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
  return emoji_pattern.sub('', text)

def remove_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def remove_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [9]:
from nltk.corpus import stopwords

def clean_text(text):
    stop = stopwords.words('arabic')
    text = remove_punctuations(text)
    text = remove_diacritics(text)
    text = remove_emojis(text)
    text = normalize_arabic(text)
    text = remove_all_entities(text)
    text = remove_links(text)
    text = ' '.join(word for word in text.split() if word not in stop)
    text = re.sub(r'[A-Za-z0-9]', '', text)
    return text

In [12]:
twitter_train_df_preprocessed = twitter_train_df.copy()
twitter_train_df_preprocessed['tweet'] = twitter_train_df_preprocessed['tweet'].apply(clean_text)

### Here we have the old Dataset : 

In [13]:
twitter_train_df

,label,tweet
0,0,اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم بالزايد 😭
1,0,توقعت اذا جات داريا بشوفهم كاملين بس لي للحين احس فيه احد ناقصهم 💔 #Avlu
2,0,#الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال والاهلي تحت التاق 👇 #تحدي_اسرع_روقان وادخل في سحب قيمة ايفون X على…
3,0,نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين على بكتيريا 🦠 فتنفجر 💥 و تموت . الأخيرة يبدو انها بكتيريا مقاومة فأخذ…
4,0,الدودو جايه تكمل علي 💔
...,...,...
46995,1,السحب الليلة على الايفون .. رتويت للمرفقة وطبق الشروط 👇
46996,1,😂 لابسة احمر ليه يا ست انتي ايه المناسبة 😂
46997,1,كلاام جمييل تستاهل(من احبه الله جعل محبته ف قلوب البشر) 💙
46998,1,- ألطف صورة ممكن تعبر عن رمضان 💙


### And the new Dataset looks like this

In [14]:
twitter_train_df_preprocessed

,label,tweet
0,0,اعترف ان بتس كانو شوي شوي يجيبو راسي اليوم بالزايد
1,0,توقعت اذا جات داريا بشوفهم كاملين للحين احس احد ناقصهم
2,0,الهلال اكتب توقعك لنتيجه لقاء الهلال والاهلي التاق اسرع روقان وادخل سحب قيمه ايفون علي…
3,0,نعمه المضادات الحيويه تضع قطرهمضاد بنسلين علي بكتيريا 🦠 فتنفجر تموت الاخيره يبدو انها بكتيريا مقاومه فاخذ…
4,0,الدودو جايه تكمل علي
...,...,...
46995,1,السحب الليله علي الايفون رتويت للمرفقه وطبق الشروط
46996,1,لابسه احمر ليه انتي ايه المناسبه
46997,1,كلاام جمييل تستاهل احبه الله محبته قلوب البشر
46998,1,الطف صوره ممكن تعبر رمضان


### Importing the test set and applying modifications on it

In [15]:
test_pos = pd.read_csv("../twitter/test_Arabic_tweets_positive_20190413.tsv", 
                       sep="\t", header=None,  quoting=csv.QUOTE_NONE)
test_neg = pd.read_csv("../twitter/test_Arabic_tweets_negative_20190413.tsv", 
                       sep="\t", header=None,  quoting=csv.QUOTE_NONE)
test_pos.rename(columns={0:'label', 1:'tweet'}, inplace=True)
test_neg.rename(columns={0:'label', 1:'tweet'}, inplace=True)
test_neg['label']=0
test_pos['label']=1
twitter_test_df = pd.concat([test_neg, test_pos], axis=0).reset_index(drop=True)
twitter_test_df_preprocessed = twitter_test_df.copy()
twitter_test_df_preprocessed['tweet'] = twitter_test_df_preprocessed['tweet'].apply(clean_text)

In [16]:
twitter_test_df

,label,tweet
0,0,حتى الايتونز خربتوه مو صاحين انتو؟؟ 😭
1,0,واحد تبع النظام السوري يقول أن المخابرات السورية وراء تحطم مركبة إسرائيلية على سطح القمر 😳 أول مرة أعرف أن القمر أق…
2,0,الى متى التعامل السئ للخادمات وعدم احترامهم وكأنهم حشرات والله هالمواقف تصير بيننا ونشوفها المربية جات من بيتها مع…
3,0,رايح جاي ي طحلبي 🐸 #الهلال_الاهلي
4,0,تتمغط ومعها سداع 😫
...,...,...
11746,1,ربي اغفر لي و لوالدي و لأحبتي و للمؤمنين و المؤمنات و المسلمين و المسلمات الأحياء منهم و الأموات :)
11747,1,ربي يسعدنا وياكم 💛
11748,1,يتحدثون عن اخلاق حسين ونجوم فرقهم نهاياتهم الرياضية أليمة ومخجلة نختلف ونتفق حول حسين ولكن المؤكد أن صحيفته الأخلاق…
11749,1,صباحكم احتفالية لم تكتمل، وصاحب الاحتفاليه ماكمل المباراة برضوه..اجل بتغبن جمهور الهلال 😂


In [17]:
twitter_test_df_preprocessed

,label,tweet
0,0,حتي الايتونز خربتوه مو صاحين انتو؟؟
1,0,تبع النظام السوري يقول ان المخابرات السوريه وراء تحطم مركبه اسراءيليه علي سطح القمر اول مره اعرف ان القمر اق…
2,0,الي متي التعامل السء للخادمات وعدم احترامهم وكانهم حشرات والله هالمواقف تصير بيننا ونشوفها المربيه جات بيتها مع…
3,0,رايح جاي طحلبي الاهلي
4,0,تتمغط ومعها سداع
...,...,...
11746,1,ربي اغفر لوالدي لاحبتي للمءمنين المءمنات المسلمين المسلمات الاحياء منهم الاموات
11747,1,ربي يسعدنا وياكم
11748,1,يتحدثون اخلاق حسين ونجوم فرقهم نهاياتهم الرياضيه اليمه ومخجله نختلف ونتفق حول حسين المءكد ان صحيفته الاخلاق…
11749,1,صباحكم احتفاليه تكتمل، وصاحب الاحتفاليه ماكمل المباراه برضوه اجل بتغبن جمهور الهلال


### Splitting the data

In [18]:
from sklearn.model_selection import train_test_split
#old train split
X = twitter_train_df.tweet.values
y = twitter_train_df.label.values

# The train val split is used by the DL approach but not classical ML
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.1, random_state=1111)

#old test split
X_test = twitter_test_df.tweet.values
y_test = twitter_test_df.label.values


### Old model definition

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import StandardScaler

def train_model(model, data, targets):
    text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', model),
    ])
    text_clf.fit(data, targets)
    return text_clf
def get_accuracy(trained_model,X, y):
    predicted = trained_model.predict(X)
    accuracy = np.mean(predicted == y)
    return accuracy

### Old model metrics

In [21]:
old_model = train_model(MultinomialNB(), X_train, y_train)
test_accuracy = get_accuracy(old_model,X_test, y_test)
print(f"test accuracy with MultinomialNB: {test_accuracy:.4f}")

test accuracy with MultinomialNB: 0.7822


### Since we are using Cross validation we can merge the two dataset(train and test) and use it as a training set for the model

In [22]:
new_twitter_df =  twitter_train_df_preprocessed.append(twitter_test_df_preprocessed, ignore_index=True)
new_twitter_df

,label,tweet
0,0,اعترف ان بتس كانو شوي شوي يجيبو راسي اليوم بالزايد
1,0,توقعت اذا جات داريا بشوفهم كاملين للحين احس احد ناقصهم
2,0,الهلال اكتب توقعك لنتيجه لقاء الهلال والاهلي التاق اسرع روقان وادخل سحب قيمه ايفون علي…
3,0,نعمه المضادات الحيويه تضع قطرهمضاد بنسلين علي بكتيريا 🦠 فتنفجر تموت الاخيره يبدو انها بكتيريا مقاومه فاخذ…
4,0,الدودو جايه تكمل علي
...,...,...
58746,1,ربي اغفر لوالدي لاحبتي للمءمنين المءمنات المسلمين المسلمات الاحياء منهم الاموات
58747,1,ربي يسعدنا وياكم
58748,1,يتحدثون اخلاق حسين ونجوم فرقهم نهاياتهم الرياضيه اليمه ومخجله نختلف ونتفق حول حسين المءكد ان صحيفته الاخلاق…
58749,1,صباحكم احتفاليه تكتمل، وصاحب الاحتفاليه ماكمل المباراه برضوه اجل بتغبن جمهور الهلال


### Creating the new pipeline with a RandomizedSearchCV object in order to get the best parameters for the pipeline

In [23]:
from sklearn.pipeline import Pipeline
from scipy.special import expit
from sklearn.preprocessing import FunctionTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score

kf = KFold(n_splits=10, shuffle=True, random_state=1111)

def Optimize_model(model, data, target):
    pipe = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('model', model)    
    ])
    params = {
    'tfidf__max_df': (0.5, 0.75, 1.0),
    'tfidf__max_features': (None, 5000, 10000, 50000),
    'tfidf__ngram_range': ((1, 1), (1, 2), (1,3)),  
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2', None),
    'model__alpha': np.linspace(0.00001, 1, 20)
    }
    search = RandomizedSearchCV(estimator=pipe, param_distributions=params, cv=kf, 
                                n_iter=100, verbose=1, n_jobs = -1)
    search.fit(data, target)
    print(search.best_params_)
    return search

In [ ]:
better_model = Optimize_model(MultinomialNB(), new_twitter_df.tweet, new_twitter_df.label)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


In [ ]:
better_model.best_score_

In [51]:
better_model.best_estimator_

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.5, ngram_range=(1, 3),
                                 use_idf=False)),
                ('model', MultinomialNB(alpha=0.26316526315789474))])

In [ ]:
# best model running 10 iterations of 1000 fits :
# {'tfidf__use_idf': False, 'tfidf__norm': 'l2', 'tfidf__ngram_range': (1, 3), 'tfidf__max_features': None, 'tfidf__max_df': 0.5, 'model__alpha': 0.26316526315789474}


In [52]:
test_accuracy = get_accuracy(better_model,twitter_test_df_preprocessed.tweet.values, twitter_test_df_preprocessed.label.values)
print(f"test accuracy with new model: {test_accuracy:.4f}")

test accuracy with new model: 0.9734


### Uploading the SS2030 dataset

In [53]:
uploaded = files.upload()

Saving Arabic Sentiment Analysis Dataset - SS2030.csv to Arabic Sentiment Analysis Dataset - SS2030.csv


In [54]:
df_ss2030 = pd.read_csv("SS2030/Arabic Sentiment Analysis Dataset - SS2030.csv")
# Rename columns to match convention
df_ss2030 = df_ss2030.rename(columns = {"text":"tweet", "Sentiment": "label"})

In [55]:
df_ss2030

,tweet,label
0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,1
1,RT @___IHAVENOIDEA: حقوق المرأة في الإسلام. https://t.co/ps3qNw1CbB,1
2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (@Shubratanmyeh):\n \n ما زال التسجيل مستمر في دورة حقوق المرأة بعد الطلاق ✨ #وعيك_يحميك... https://t.co/c2NXzNCdLU,1
3,RT @MojKsa: حقوق المرأة التي تضمنها لها وزارة العدل https://t.co/QUGzWwubFk,1
4,RT @abm112211: ولي امر الزوجة او ولي الزوجة او ولي المراة من الاخطاء الشائعة \n \n هذا الكلام غلط في الشريعة والقانون\n فلا يوجد ولي للزوجة او المراة الا اذا كانت قاصرا ويكون الولي ابوها ...الخ وليس الزوج منهم\n نعم له حقوق عليها لكنها ليست ولاية\n الولاية مصطلح فقهي قانوني ...,1
...,...,...
4247,#غرد_بحبك_لمحمد_بن_سلمان ❤️,1
4248,#غرد_بحبك_لمحمد_بن_سلمان \n محمدبن سلمان احبه الله واختاره في هذا لوقت لشيءً هو يعلمه اما حبنا له فهو طبيعي ان يكون فوق العاده لاننا في حاجة لمثل هذا الرجل الملهم البطل المقدام حفظه الله وحفظ به امن وامان بلاد الحرمين الشريفين وامن الشعب السعودي الوفي,1
4249,#غرد_بحبك_لمحمد_بن_سلمان \n الله يحفظك يا ذخر الوطن ويخليك 💜,1
4250,#غرد_بحبك_لمحمد_بن_سلمان \n \n الله يحفظه ويحميه ويقويه وكلنا معه 💚,1


### Applying the modifications to the SS2030 dataset

In [56]:
df_ss2030_preprocessed = df_ss2030.copy()
df_ss2030_preprocessed['tweet'] = df_ss2030_preprocessed['tweet'].apply(clean_text)
df_ss2030_preprocessed

,tweet,label
0,حقوق المراه,1
1,حقوق المراه الاسلام,1
2,لجنه التنميه بشبرا زال التسجيل مستمر دوره حقوق المراه الطلاق يحميك,1
3,حقوق المراه تضمنها وزاره العدل,1
4,ولي امر الزوجه او ولي الزوجه او ولي المراه الاخطاء الشاءعه الكلام غلط الشريعه والقانون يوجد ولي للزوجه او المراه الا اذا كانت قاصرا ويكون الولي ابوها الخ وليس الزوج منهم حقوق عليها لكنها ولايه الولايه مصطلح فقهي قانوني محدد,1
...,...,...
4247,بحبك لمحمد بن سلمان,1
4248,بحبك لمحمد بن سلمان محمدبن سلمان احبه الله واختاره لوقت لشيء يعلمه اما حبنا فهو طبيعي ان يكون العاده لاننا حاجه لمثل الرجل الملهم البطل المقدام حفظه الله وحفظ امن وامان بلاد الحرمين الشريفين وامن الشعب السعودي الوفي,1
4249,بحبك لمحمد بن سلمان الله يحفظك ذخر الوطن ويخليك,1
4250,بحبك لمحمد بن سلمان الله يحفظه ويحميه ويقويه وكلنا معه,1


### Evaluating the new model vs the old model

In [58]:
ss_2030_accuracy = get_accuracy(old_model,df_ss2030.tweet.values, df_ss2030.label.values)
print(f"df_ss2030 dataset accuracy with Multinomial NB: {ss_2030_accuracy:.4f}")
SS_2030_acc = get_accuracy(better_model,df_ss2030_preprocessed.tweet.values, df_ss2030_preprocessed.label.values)
print(f"df_ss2030_enhanced dataset accuracy with New model: {SS_2030_acc:.4f}")

df_ss2030 dataset accuracy with Multinomial NB: 0.5910
df_ss2030_enhanced dataset accuracy with New model: 0.6150


### Uploading the arabic 100K review

In [61]:
uploaded = files.upload()

Saving ar_reviews_100k.tsv to ar_reviews_100k.tsv


In [62]:
df_reviews = pd.read_csv("ar_reviews_100k.tsv", delimiter="\t")
# Create a mapping for the labels such that we use the same convention across all datasets
label_mapping = {"Positive": 1, "Negative":0}
# Filter to only have pos and neg tweets, i.e: remove mixed tweets
df_reviews = df_reviews[df_reviews.label != "Mixed"]
df_reviews["label"] = df_reviews["label"].map(label_mapping)
# Rename columns to match convention
df_reviews = df_reviews.rename(columns = {"text":"tweet"})

In [63]:
df_reviews

,label,tweet
0,1,ممتاز نوعا ما . النظافة والموقع والتجهيز والشاطيء. المطعم
1,1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدولة يعشق ترابها. نحن نحب الإمارات. ومضات من فكر. نصائح لدولة تطمح بالصفوف الأولى و قائد لا يقبل إلا براحة شعبه وتوفر كل سب العيش الكريم. حكم و مواقف ونصائح لكل فرد فينا ليس بمجرد كتاب سياسي كما كنت اعتقد. يستحق القراءة مرات كثيرة
2,1,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى هدوء جبال الشيشان .. للتعرف على حقيقة ما يجرى فى تلك البلاد من حروب ضاربة بحق المسلمين و جزء كبير من تاريخ تلك المنطقة. التضحية .. الرجولة .. الوفاء والكثير من القيم الأخرى اثبتت وجودها فى تلك الرواية البسيطة
3,1,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الازرق ميقراش احسن.. احمد مراد تخطى مرحلة ان القارئ يخلص الرواية وهو فاتح بؤه لمرحلة ان القارئ يخلص الرواية وهو محترم الكاتب.. اتقان مخيف.. بصرف النظر عن اخطاء لا تذكر ف الحوار.. انما احمد مراد سافر عاش حبة ف اوائل القرن العشرين وجه ي...
4,1,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل الخدمات مريح نفسيا. لا يوجد
...,...,...
99994,0,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من البداية..القصة تقليدية، الاحداث بطيئة ومملة والرواي أطول مما تستوجب.... وصلت لبعد منتصفها وقررت إنها متستحقش أضيع وقت تاني فيها
99995,0,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا يوجد خدمة افطار صباحي مستوي الفندق غير لائق
99996,0,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحاله أو مشهد بدون فكره للقصه
99997,0,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف عندة دقيق وزائد عن حد اللزوم.. هذا ثاني كتاب اقراءة للكتاب على أمل اني احب كتابته، لكن للأسف كان سيء زي الاول.


### Applying the feature engineering on the 100K dataset

In [64]:
df_reviews_preprocessed = df_reviews.copy()
df_reviews_preprocessed['tweet'] = df_reviews_preprocessed['tweet'].apply(clean_text)
df_reviews_preprocessed

,label,tweet
0,1,ممتاز نوعا النظافه والموقع والتجهيز والشاطيء المطعم
1,1,احد اسباب نجاح الامارات ان شخص الدوله يعشق ترابها نحب الامارات ومضات فكر نصاءح لدوله تطمح بالصفوف الاولي قاءد يقبل الا براحه شعبه وتوفر سب العيش الكريم حكم مواقف ونصاءح لكل فرد فينا بمجرد كتاب سياسي كنت اعتقد يستحق القراءه مرات كثيره
2,1,هادفه وقويه تنقلك صخب شوارع القاهره الي هدوء جبال الشيشان للتعرف علي حقيقه يجري البلاد حروب ضاربه بحق المسلمين جزء كبير تاريخ المنطقه التضحيه الرجوله الوفاء والكثير القيم الاخري اثبتت وجودها الروايه البسيطه
3,1,خلصنا مبدءيا اللي مستني ابهار زي الفيل الازرق ميقراش احسن احمد مراد تخطي مرحله ان القارء يخلص الروايه فاتح بءه لمرحله ان القارء يخلص الروايه محترم الكاتب اتقان مخيف بصرف النظر اخطاء تذكر الحوار انما احمد مراد سافر عاش حبه اواءل القرن العشرين وجه ياخدنا لهناك خلطه مشاعر انساني...
4,1,ياسات جلوريا جزء يتجزا دبي فندق متكامل الخدمات مريح نفسيا يوجد
...,...,...
99994,0,معرفش ليه كنت عاوزه اكملها وهي مش عاجباني البدايه القصه تقليديه، الاحداث بطيءه وممله والرواي اطول تستوجب وصلت لبعد منتصفها وقررت انها متستحقش اضيع وقت تاني
99995,0,يستحق ان يكون بوكنق لانه سيء شي يوجد خدمه افطار صباحي مستوي الفندق لاءق
99996,0,كتاب ضعيف جدا ولم استمتع قصه سرد لحاله او مشهد بدون فكره للقصه
99997,0,ممله جدا محمد حسن علوان فنان بالكلمات، والوصف عنده دقيق وزاءد حد اللزوم كتاب اقراءه للكتاب علي امل اني احب كتابته، للاسف سيء زي الاول


### Evaluating the accuracy of the two models

In [65]:
df_reviews_accuracy = get_accuracy(old_model,df_reviews.tweet.values, df_reviews.label.values)
print(f"df_reviews dataset accuracy with Multinomial NB: {df_reviews_accuracy:.4f}")
df_reviews_new_accuracy = get_accuracy(better_model,df_reviews_preprocessed.tweet.values, df_reviews_preprocessed.label.values)
print(f"df_reviews_clean dataset accuracy with New model: {df_reviews_new_accuracy:.4f}")

df_reviews dataset accuracy with Multinomial NB: 0.6007
df_reviews_clean dataset accuracy with New model: 0.6133


### Uploading ArSAS 

In [66]:
uploaded = files.upload()

Saving ArSAS.txt to ArSAS.txt


In [67]:
df_arsas = pd.read_csv('ArSAS.txt', header = 0, delimiter = "\t")
# Filter to only have pos and neg tweets, i.e: remove mixed tweets
df_arsas_pos = df_arsas[df_arsas.Sentiment_label == 'Positive'] 
df_arsas_neg = df_arsas[df_arsas.Sentiment_label == 'Negative'] 
df_arsas = pd.concat([df_arsas_pos, df_arsas_neg], axis=0).reset_index(drop=True)
# Create a mapping for the labels such that we use the same convention across all datasets
label_mapping = {"Positive": int(1), "Negative":int(0)}
df_arsas["Sentiment_label"] = df_arsas["Sentiment_label"].map(label_mapping)
# Rename columns to match convention
df_arsas = df_arsas.rename(columns = {"Tweet_text":"tweet", "Sentiment_label":"label"})

In [68]:
df_arsas

,#Tweet_ID,tweet,Topic,label,Sentiment_label_confidence,Speech_act_label,Speech_act_label_confidence
0,929241870508724224,المباراة القـادمة #غانا x #مصر الجولة الأخيرة من المجموعة الـ 5 تصفيات كاس العالم 2018 روسـيا ترتيب مصر : المركز الاول 12 نقطة ( تم حسم التأهل للمونديال ) غــدا الساعة 5:30 ع قناة : بين ســبورت 1 تـــوقعاتكم لـ نتيجة الماتش .؟ 😀😁 https://t.co/RTQBNZXDqM,Event,1,0.38,Assertion,0.62
1,928615163250520065,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت بارتياح وأنا أتابعه من باريس - https://t.co/hSvsbEaeUz #youm,Event,1,0.69,Assertion,1.00
2,929607749461250048,بسم الله نبدأ 👏 نغرد علي وسم 👇 👇 👇 👇 👇 ↩ #شباب_مصر_يريد ↪تحقيق أهداف ثورة يناير العيش والحرية والعدالة الاجتماعية والكرامة الإنسانية #تيم_المرابطون ✊ https://t.co/RDDVULxYuK,Long_Standing,1,1.00,Expression,1.00
3,928932760017866754,رحم الله شهداء ثورة يناير اللي بسببهم اتكسر الخوف جوانا من بلطجية الداخلية وبقو بالنسبالنا موظف عادي في الدولة,Long_Standing,1,0.35,Expression,1.00
4,929377209441284096,ليلة كروية أفريقية حاسمة في #تصفيات_كأس_العالم لأفريقيا، وكل التمنيات بالتوفيق لكل مم #المغرب و #تونس اللي يكفيهم التعادل للصعود. -- كوت ديفوار والمغرب .. 7:30 م تونس وليبيا .. 8:30 م -- https://t.co/KEZhIRxREP,Event,1,0.66,Assertion,0.67
...,...,...,...,...,...,...,...
11779,927250764514430977,على قولت مرتضى منصور باكابورت ضارب ... 😏,Entity,0,1.00,Expression,1.00
11780,927930867271524353,ماشى اوكى هترجع تيران وصنافير وهتفرج عن المعتقلين وبعدين؟!طيب والإرهاب والاسعار والتعليم والصحة والسياحة والاستثمار والاقتصاد مش هما الاولى,Long_Standing,0,0.66,Expression,1.00
11781,929273539781328896,انت ودولتكم سرطان هذه الامه نسأل العظيم ان يرد كيدكم في نحوركم ويخرج ذنب الدماء اللي قتلت باسبب تدخلتكم في زعزعة واستقرار شعوب الربيع العربي وقتل الالف في رابعه واليمن وليبيا الله جبار منتقم وتلك الايام ندولها بين الناس,Long_Standing,0,1.00,Expression,1.00
11782,927902747223044097,أزمة المثقف العربي.. في ظل إشراقات”الربيع العربي” https://t.co/561tHwc3CO https://t.co/urHQlHljsN,Long_Standing,0,0.69,Assertion,0.69


In [70]:
df_arsas_preprocessed = df_arsas.copy()
df_arsas_preprocessed = df_arsas_preprocessed.drop(columns=['#Tweet_ID','Topic','Sentiment_label_confidence','Speech_act_label','Speech_act_label_confidence'], axis=1)
df_arsas_preprocessed['tweet'] = df_arsas_preprocessed['tweet'].apply(clean_text)
df_arsas_preprocessed

,tweet,label
0,المباراه القادمه الجوله الاخيره المجموعه ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطه تم حسم التاهل للمونديال الساعه قناه سبورت توقعاتكم نتيجه الماتش ؟,1
1,وزير خارجيه فرنسا منتدي شباب العالم شعرت بارتياح وانا اتابعه باريس,1
2,بسم الله نبدا نغرد علي وسم ↩ مصر يريد ↪تحقيق اهداف ثوره العيش والحريه والعداله الاجتماعيه والكرامه الانسانيه المرابطون,1
3,رحم الله شهداء ثوره اللي بسببهم اتكسر الخوف جوانا بلطجيه الداخليه وبقو بالنسبالنا موظف عادي الدوله,1
4,ليله كرويه افريقيه حاسمه كاس العالم لافريقيا، وكل التمنيات بالتوفيق لكل مم اللي يكفيهم التعادل للصعود كوت ديفوار والمغرب تونس وليبيا,1
...,...,...
11779,علي قولت مرتضي منصور باكابورت ضارب,0
11780,ماشي اوكي هترجع تيران وصنافير وهتفرج المعتقلين وبعدين؟ طيب والارهاب والاسعار والتعليم والصحه والسياحه والاستثمار والاقتصاد مش الاولي,0
11781,انت ودولتكم سرطان الامه نسال العظيم ان يرد كيدكم نحوركم ويخرج ذنب الدماء اللي قتلت باسبب تدخلتكم زعزعه واستقرار شعوب الربيع العربي وقتل الالف رابعه واليمن وليبيا الله جبار منتقم وتلك الايام ندولها الناس,0
11782,ازمه المثقف العربي ظل اشراقات”الربيع العربي”,0


### Evaluating the models on the df_arsas :

In [73]:
df_arsas_accuracy = get_accuracy(old_model,df_arsas.tweet.values, df_arsas.label.values)
print(f"df_arsas dataset accuracy with Multinomial NB: {df_arsas_accuracy:.4f}")
df_arsas_new_accuracy = get_accuracy(better_model,df_arsas_preprocessed.tweet.values, df_arsas_preprocessed.label.values)
print(f"df_arsas_clean dataset accuracy with New model: {df_arsas_new_accuracy:.4f}")

df_arsas dataset accuracy with Multinomial NB: 0.6530
df_arsas_clean dataset accuracy with New model: 0.6461
